# One-class SVM の改善検討

scikit-learn の <b>One-class SVM</b> のアノマリー検出性能を改善するための取り組みになります。

試しに今回は「する」をストップワードにしてから再検証してみましたところ、全件がアノマリーとして検出されたことが確認できました。

## (1) テストデータ／環境準備

マイオペで使用しているテストデータ（learning/tests/engine/fixtures/ 配下のCSVファイル）をベースに動作確認を行います。

動作確認にあたっては、MySQLdb に接続できないため、ローカル環境テスト用の Bot クラスを使用しています。

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_benefitone_conversation.csv',
    'test_daikin_conversation.csv',
    'test_ptna_conversation.csv',
    'test_septeni_conversation.csv'
]
copied_csv_file_paths = TestTool.copy_testdata_csv(learning_dir, csv_file_names)
copied_csv_file_paths

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.csv]


['/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv',
 '/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv',
 '/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv',
 '/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.csv']

## (2) TF-IDFベクターの準備

Bot クラス内に組み込まれている __build_training_set_from_csv 関数をバラして実行しています。

In [3]:
'''
    初期設定
    データファイル、エンコードを指定
    内容は、learn.py を参考にしました。    
'''
from learning.core.learn.learning_parameter import LearningParameter
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': None,
    # 'algorithm': LearningParameter.ALGORITHM_NAIVE_BAYES
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    # 'params_for_algorithm': { 'C': 200 }
    'params_for_algorithm': {}
}
learning_parameter = LearningParameter(attr)

bot_id = 7777
csv_file_encoding = 'utf-8'

### (2-1) 訓練データのTF-IDFベクター

In [4]:
'''
    訓練データの生成（内部で TF-IDF 処理を実行）
    
    text_array.py における TF-IDF 処理では、
    以下の通り「する」をストップワード化指定しております
    
    class TextArray:
        :
        def __build_vectorizer(self):
            :
            vectorizer = TfidfVectorizer(use_idf=False, token_pattern=u'(?u)\\b\\w+\\b', stop_words=['する'])
'''
#from learning.core.training_set.training_message_from_csv import TrainingMessageFromCsv
from prototype.modules.training_message_from_csv import TrainingMessageFromCsv
training_set = TrainingMessageFromCsv(bot_id, copied_csv_file_paths, learning_parameter, encoding=csv_file_encoding)
build_training_set_from_csv = training_set.build()

X = build_training_set_from_csv.x
y = build_training_set_from_csv.y

2017/03/22 PM 01:05:26 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 28272
2017/03/22 PM 01:05:26 TextArray#__init__ start
2017/03/22 PM 01:05:41 TextArray#to_vec start
2017/03/22 PM 01:05:41 TextArray#to_vec end


In [5]:
n_sample = X.shape[0]
n_feature = X.shape[1]
print("sample=%d, feature=%d" % (n_sample, n_feature))

sample=28272, feature=2748


### (2-2) 単語の頻出度合いをチェック

In [6]:
text_array = build_training_set_from_csv._body_array
vocabulary = text_array._vectorizer.vocabulary_

#### featureが抽出されたサンプルの件数を、featureごとに集計します。

In [7]:
ext_feature_count = [0.0 for i in range(n_feature)]
tot_feature_count = 0.0
for i in range(n_sample):
    
    arr = X[i].toarray()[0]
    for j in range(n_feature):
        if arr[j] == 0.0:
            continue
        ext_feature_count[j] += 1.0
        tot_feature_count += 1.0

#### 降順ソートして表示（上位１０件）

In [8]:
ext_feature_list = []
for k, v in enumerate(ext_feature_count):
    item = TestTool.get_item_from_vocabulary(vocabulary, k)
    elem = (item, v, v / tot_feature_count)
    ext_feature_list.append(elem)

sorted_list = sorted(ext_feature_list, key=lambda x: x[1], reverse=True)

for i in range(50):
    print("[%s] count=%d, rate=%0.4f" % sorted_list[i])

[メール] count=4900, rate=0.0263
[できる] count=3516, rate=0.0189
[れる] count=3379, rate=0.0181
[2010] count=2694, rate=0.0145
[outlook] count=2477, rate=0.0133
[教える] count=2394, rate=0.0129
[暗号] count=2252, rate=0.0121
[設定] count=2100, rate=0.0113
[表示] count=1973, rate=0.0106
[方法] count=1856, rate=0.0100
[e] count=1769, rate=0.0095
[インストール] count=1766, rate=0.0095
[管理] count=1720, rate=0.0092
[利用] count=1624, rate=0.0087
[どう] count=1508, rate=0.0081
[ツール] count=1506, rate=0.0081
[化] count=1414, rate=0.0076
[システム] count=1408, rate=0.0076
[dki] count=1404, rate=0.0075
[it] count=1328, rate=0.0071
[知る] count=1319, rate=0.0071
[資産] count=1302, rate=0.0070
[パスワード] count=1262, rate=0.0068
[なる] count=1236, rate=0.0066
[使用] count=1234, rate=0.0066
[pointsec] count=1226, rate=0.0066
[いい] count=1221, rate=0.0066
[端末] count=1050, rate=0.0056
[可能] count=1006, rate=0.0054
[ファイル] count=1002, rate=0.0054
[欲しい] count=994, rate=0.0053
[出来る] count=968, rate=0.0052
[アウト] count=932, rate=0.0050
[先生] count=914, 

### (2-3) 外れデータのTF-IDFベクター

比較のため、<b><a href="12.ipynb">こちらで再検証した時と同じ質問文</a></b>をつかっております。

In [9]:
'''
    マイオペのプロダクション・コードと同じように、
    訓練データ作成時と同じベクトライザーを使用します。
'''
#from learning.core.training_set.text_array import TextArray
from prototype.modules.text_array import TextArray

test_X = [
    '要素技術は自然languageの機械learningですか？', # まったくfeatureが抽出されない質問文

    '人生相談をしたいのですが？', # featureが１件抽出されるな質問文「する」を含む
    '難解なプログラミング技術を調達？', # featureが１件抽出される質問文「する」を含まない

    '何か習い事をしますか？', # featureが２件抽出される質問文「する」を含む
    '何か習い事がいいですか？', # featureが２件抽出される質問文「する」を含まない
    
    '何か習い事をしたほうがいいですか？', # featureが３件抽出される質問文「する」を含む
    '何か習い事を行うほうがいいですか？', # featureが３件抽出される質問文「する」を含まない
    
    '会社を辞めたいのですが誰に相談するのがいいですか？', # featureが４件抽出される質問文「する」を含む
    '会社を辞めたいのですが誰に相談を行うのがいいですか？', # featureが４件抽出される質問文「する」を含まない

    '有給休暇を取って海外に行き旅行する意向があります。', # featureが５件抽出される質問文「する」を含む
    '有給休暇を取って海外に行きたいと思っています。', # featureが５件抽出される質問文「する」を含まない
]
vectorizer = training_set.body_array.vectorizer
text_array = TextArray(test_X, vectorizer=vectorizer)

'''
    外れデータのTF-IDFベクターを取得
'''
X_error = text_array.to_vec()

2017/03/22 PM 01:05:59 TextArray#__init__ start
2017/03/22 PM 01:05:59 TextArray#to_vec start
2017/03/22 PM 01:05:59 TextArray#to_vec end


In [10]:
vocabulary = text_array._vectorizer.vocabulary_
dumped_features = TestTool.get_dumped_features(X_error, vocabulary)
for d in dumped_features:
    print(d)

index=0[]
index=1[]
index=2[調達=1.000]
index=3[何=1.000]
index=4[いい=0.707 何=0.707]
index=5[いい=0.707 何=0.707]
index=6[いい=0.577 何=0.577 行う=0.577]
index=7[いい=0.577 会社=0.577 誰=0.577]
index=8[いい=0.500 会社=0.500 行う=0.500 誰=0.500]
index=9[休暇=0.500 取る=0.500 海外=0.500 行く=0.500]
index=10[休暇=0.447 取る=0.447 思う=0.447 海外=0.447 行く=0.447]


## (3) One-class SVMで学習

線形カーネルを使用しています。

In [11]:
'''
    One-class SVM with linear kernel, anomary is half
        fn(i,j) = i'*j
'''
from sklearn import svm
clf_linear = svm.OneClassSVM(
    kernel='linear', # linearカーネルを使用
    nu=0.5           # 外れ判定される件数の見積もりを全体の5割と想定
    ) 
clf_linear.fit(X)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto',
      kernel='linear', max_iter=-1, nu=0.5, random_state=None,
      shrinking=True, tol=0.001, verbose=False)

## (4) 外れデータを使って予測

全件がアノマリーとして検出されたことが確認できました。

In [12]:
'''
    外れデータを使用して、外れ判定を実行してみます
'''
y_pred_by_linear = clf_linear.predict(X_error)
y_pred_by_linear

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])